In [1]:
%%writefile .devcontainer/.dockerignore
**/.git
**/.vscode
**/.idea
**/__pycache__
**/*.pyc
**/*.pyo
**/*.pyd
**/*.swp
**/venv
**/env



Overwriting .devcontainer/.dockerignore


In [2]:
%%writefile .devcontainer/.env
ENV_NAME=ml_project
COMPOSE_PROJECT_NAME=ml_project
PYTHON_VER=3.10
QUARTO_VER=1.3.450
JUPYTER_PORT=8888
TENSORBOARD_PORT=6006
BUILD_TF_FROM_SOURCE=false



Overwriting .devcontainer/.env


In [3]:
%%writefile .devcontainer/.devcontainer.json
{
    "name": "data_science_example",
    "build": {
        "dockerfile": "Dockerfile",
        "context": "..",
        "args": {
            "ENV_NAME": "data_science_example",
            "PYTHON_VER": "${localEnv:PYTHON_VER:3.10}",
            "QUARTO_VER": "${localEnv:QUARTO_VER:1.3.450}"
        }
    },
    "customizations": {
        "vscode": {
            "settings": {
                "python.defaultInterpreterPath": "/opt/conda/envs/data_science_example/bin/python",
                "python.pythonPath": "/opt/conda/envs/data_science_example/bin/python",
                "terminal.integrated.profiles.linux": {
                    "bash": {
                        "path": "bash",
                        "args": ["-l"]
                    }
                },
                "terminal.integrated.defaultProfile.linux": "bash"
            },
            "extensions": [
                "quarto.quarto",
                "purocean.drawio-preview",
                "redhat.vscode-yaml",
                "yzhang.markdown-all-in-one",
                "ms-azuretools.vscode-docker",
                "ms-vscode-remote.remote-containers",
                "ms-python.python",
                "ms-toolsai.jupyter",
                "github.vscode-github-actions"
            ]
        }
    },
    "mounts": [
        "source=${localWorkspaceFolder},target=/workspace,type=bind,consistency=cached"
    ],
    "remoteEnv": {
        "MY_VAR": "${localEnv:MY_VAR:test_var}"
    },
    "runArgs": [
        "--gpus", "all",
        "--env-file", ".devcontainer/devcontainer.env"
    ],
    "postCreateCommand": "conda run -n data_science_example python /workspace/tests/test1.py"
}



Writing .devcontainer/.devcontainer.json


In [4]:
%%writefile .devcontainer/Dockerfile
# Use NVIDIA CUDA base image for GPU support
FROM nvidia/cuda:12.0.0-base-ubuntu22.04

# Define ARGs
ARG ENV_NAME=data_science_example
ARG PYTHON_VER=3.10

# Prevent tzdata from asking for user input
ENV DEBIAN_FRONTEND=noninteractive
ENV LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH
ENV LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libstdc++.so.6

# Set environment variables
ENV CONDA_DIR=/opt/conda
ENV PATH=$CONDA_DIR/bin:$PATH
ENV ENV_NAME=$ENV_NAME

# Install system dependencies
RUN apt-get update --fix-missing && apt-get install -y \
    wget \
    bzip2 \
    ca-certificates \
    curl \
    git \
    vim \
    python3-pip \
    python3-dev \
    python3-opencv \
    libglib2.0-0 \
    libstdc++6 \
    libgcc1 \
    libc6 \
    libc6-dev \
    libc-dev-bin \
    && apt-get clean \
    && rm -rf /var/lib/apt/lists/*

# Install Miniconda
RUN wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh \
    && /bin/bash ~/miniconda.sh -b -p /opt/conda \
    && rm ~/miniconda.sh \
    && $CONDA_DIR/bin/conda clean -a -y

# Create a new Conda environment
COPY .devcontainer/environment.yml /tmp/environment.yml
RUN sed -i "s/name: \${ENV_NAME}/name: $ENV_NAME/" /tmp/environment.yml && \
    sed -i "s/- python=\${PYTHON_VER}/- python=$PYTHON_VER/" /tmp/environment.yml && \
    conda env create -f /tmp/environment.yml \
    && conda clean -a -y

# Set up shell initialization for Conda
RUN echo ". /opt/conda/etc/profile.d/conda.sh && conda activate $ENV_NAME" >> /etc/bash.bashrc

# Upgrade pip and install additional packages within the Conda environment
RUN /bin/bash -c ". /opt/conda/etc/profile.d/conda.sh && conda activate $ENV_NAME && \
    pip install --upgrade pip && \
    pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118"

# Set working directory
WORKDIR /workspace

# Set the default command to bash
CMD ["/bin/bash"]





Overwriting .devcontainer/Dockerfile


In [5]:
%%writefile .devcontainer/environment.yml
name: data_science_example
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.10
  - numpy
  - pandas
  - scikit-learn
  - matplotlib
  - seaborn
  - jupyter
  - ipykernel
  - pytest
  - black
  - flake8
  - mypy
  - pip
  - seaborn
  - mysql-connector-python
  - sqlalchemy
  - pip:
    - pretty_errors 
    - gdown
    - xgboost
    - CatBoost
    - lightgbm
    - git+https://github.com/ghadfield32/ml_preprocessor
    - shap
    - scikit-optimize
    - shapash
    - bayesian-optimization


Overwriting .devcontainer/environment.yml


In [6]:
%%writefile .devcontainer/install_dependencies.sh

#!/usr/bin/env bash

# Install system dependencies
# apt-get update updates the package list.
# apt-get install installs specified packages.
# Options:
#    Add or remove packages as needed for your environment.
apt-get update && apt-get install -y --no-install-recommends \
    python3-launchpadlib \
    vim \
    && rm -rf /var/lib/apt/lists/*




Overwriting .devcontainer/install_dependencies.sh


In [7]:
%%writefile .devcontainer/install_quarto.sh
#!/usr/bin/env bash
QUARTO_VERSION=$1

echo "Installing Quarto version $QUARTO_VERSION"

# Identify the CPU type (M1 vs Intel)
# uname -m returns the machine hardware name.
# CPU variable is set based on the hardware type.
if [[ $(uname -m) ==  "aarch64" ]] ; then
  CPU="arm64"
elif [[ $(uname -m) ==  "arm64" ]] ; then
  CPU="arm64"
else
  CPU="amd64"
fi

# Download and install Quarto
# wget downloads the Quarto installation package.
# dpkg installs the downloaded package.
TEMP_QUARTO="$(mktemp)" && \
    wget -q  -O "$TEMP_QUARTO" https://github.com/quarto-dev/quarto-cli/releases/download/v$QUARTO_VERSION/quarto-${QUARTO_VERSION}-linux-${CPU}.deb && \
    dpkg -i "$TEMP_QUARTO" && \
    rm -f "$TEMP_QUARTO"


Overwriting .devcontainer/install_quarto.sh


In [8]:
%%writefile .devcontainer/install_requirements.sh
#!/usr/bin/env bash
ENV_NAME=$1
PYTHON_VER=$2
CPU=$(uname -m)

# Install Miniconda
# apt-get update updates the package list.
# wget downloads the Miniconda installer script.
# /bin/bash executes the installer script.
# Options:
#    Change the URL to download a different version of Miniconda if needed.
apt update && apt-get install -y --no-install-recommends \
    software-properties-common \
    && add-apt-repository -y ppa:deadsnakes/ppa \
    && apt update 

wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-${CPU}.sh -O ~/miniconda.sh \
    && /bin/bash ~/miniconda.sh -b -p /opt/conda \
    && export PATH=/opt/conda/bin:$PATH \
    && conda init bash \
    && conda install conda-build

# Set environment
# Create a new Conda environment and activate it.
. /root/.bashrc \
    && conda create -y --name $ENV_NAME python=$PYTHON_VER 

echo "export QUARTO_PYTHON=/opt/conda/envs/${ENV_NAME}/bin/python3" >> ~/.bashrc
echo "conda activate $ENV_NAME" >> ~/.bashrc

conda activate $ENV_NAME

# Install the Python packages
# pip3 install installs the packages listed in requirements.txt.
pip3 install -r /requirements/requirements.txt



Overwriting .devcontainer/install_requirements.sh


In [9]:
%%writefile .env
ENV_NAME=ml_project
COMPOSE_PROJECT_NAME=ml_project
PYTHON_VER=3.10
QUARTO_VER=1.3.450
JUPYTER_PORT=8888
TENSORBOARD_PORT=6006




Overwriting .env


In [10]:
%%writefile docker-compose.yml
services:
  datascience:
    build:
      context: .
      dockerfile: .devcontainer/Dockerfile
    runtime: nvidia  # Ensure the NVIDIA runtime is used
    volumes:
      - .:/workspace
    ports:
      - "8888:8888"  # For Jupyter Notebook
      - "6006:6006"  # For TensorBoard
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: all
              capabilities: [gpu]
    environment:
      - ENV_NAME=data_science_example
      - PYTHON_VER=3.10
    command: ["conda", "run", "-n", "data_science_example", "jupyter", "lab", "--ip", "0.0.0.0", "--port", "8888", "--no-browser", "--allow-root"]

    



Overwriting docker-compose.yml
